## Notes

In [ ]:
# Sectioning lines
# -------------------------------------------------

In [ ]:
# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
# These sections are encouraged to be altered! (Of course, you can change all of the code!)
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

You can save animations by running them in the HTML5 form, and downloading them manually (hover over the video player and it should show a 'download' option). I've saved the individual frames to seperate files, but haven't found a way to reanimate them.

exno and figno are the excercise number and figure number, each of which can be viewed in the original version of this file (chile2010b.ipynb)

All reference to Manning, including use of the phrase "Manning's value", "Manning's", or "Mann", refers exclusively to Manning's coefficient of friction (or roughness), also known as Manning's n. 

If you run the notebook, and don't hear your computer fan working, check there is not a 'Warning' box thrown by the program. These have been put in place to avoid overwriting data, and running uneccessary simulations. On Jupyter Lab, the screen should automatically jump to the 'Warning' box, which needs you to press the *Enter* key to continue the program. This is not the same as an error box. With a 'Warning' box, the **kernel** will be `Busy`; but with an error box, the **kernel** will be `Idle`.

# Notebook setup

In [ ]:
%pylab inline

In [ ]:
from clawpack.clawutil import nbtools
from clawpack.visclaw import animation_tools
from IPython.display import HTML, Image as ipyImage
from PIL import Image
from os import environ, path, mkdir, remove
from importlib import import_module
from shutil import rmtree, copyfile
from glob import glob
from distutils.dir_util import copy_tree
import decimal

In [ ]:
print("CLAW =", environ["CLAW"])
print("FC =", environ["FC"])

In [ ]:
nbtools.make_exe(new=True, verbose=True)

In [ ]:
run maketopo.py  # download the topo file and create the dtopo file

Make save paths if not found.

In [ ]:
save_paths = "_saved", "_saved/_plots", "_saved/_anim"
for targ in save_paths:
    if path.isdir(targ) == False:
        mkdir(targ)

Clear old data files and diretories

In [ ]:
for k in range(1, 11):
    try:
        remove("setrun_{}.py".format(k))
        remove("plot_output_{}.txt".format(k))
        remove("run_output_{}.txt".format(k))
        rmtree("_plots_{}".format(k))
        rmtree("_output_{}".format(k))
    except Exception:
        pass

# Functions

In [ ]:
def old_clear():
    old_data = "setrun_{}.py", "plot_output_{}.txt", "run_output_{}.txt"
    old_paths = "_plots_{}", "_output_{}"
    for k in range(1, 11):
        for targ in old_data:
            if path.isfile(targ.format(k)) == True:
                remove(targ.format(k))
        for targ in old_paths:
            if path.isdir(targ.format(k)) == True:
                rmtree(targ.format(k))

## Initiate data compilation

Initiate changes to be written to the .data files

In [ ]:
def init_comp(
    exno,
    anim123=False,
    plots=False,
    gauge_map=False,
    gauge_type="stationary",
    out=False,
):

    comp_manns = range(1, amount_manns + 1) if plots == True else anim_manns
    # -------------------------------------------------
    old_clear()
    print("Loading...")
    # -------------------------------------------------
    for element in comp_manns:
        copyfile("setrun.py", "setrun_{}.py".format(element))
        mod = importlib.import_module("setrun_{}".format(element))
        rundata = mod.setrun()
        # Changes to the original setrun.py
        # -------------------------------------------------
        # Change Manning's
        rundata.geo_data.manning_coefficient = manns[element - 1]
        # Change output times:
        rundata.clawdata.output_style = (
            exno  # This is the difference from ex1 to ex2 in the animation
        )
        rundata.clawdata.output_times = linspace(3.5, 8, 19) * 3600.0
        # Change gauge location:
        from clawpack.amrclaw.data import GaugeData

        rundata.replace_data("gaugedata", GaugeData())  # clear old gauge data
        rundata.gaugedata.gauges = []  # empty list
        rundata.gaugedata.gtype = gauge_type
        # -------------------------------------------------
        gplot_amount = 2 if anim123 == True else len(dict_gauge)
        for j in range(gplot_amount):
            rundata.gaugedata.gauges.append(
                [det[j][0], det[j][1], det[j][2], det[j][3], det[j][4]]
            )  # not currently close to epicentre gauge
        if gauge_map == True:
            if element != comp_manns[0]:
                return
            rundata.amrdata.amr_levels_max = 3
            rundata.amrdata.refinement_ratios_x = [2, 2]
            rundata.amrdata.refinement_ratios_y = [2, 2]
            rundata.amrdata.refinement_ratios_t = [2, 2]
            rundata.regiondata.regions = []  # empty list of regions
            rundata.regiondata.regions.append([3, 3, 0.0, 1e9, -120, -60, -60, 0])
            # now comp_manns has length of 1, so user will see '100% complete'
            comp_manns = [1]
        # -------------------------------------------------
        compiler(rundata, element, comp_manns, out)

## Compile data

Write (any changes made by the user to) .data files, and then run the code from these files to create fortran data and images to be used in animation. 

In [ ]:
def compiler(rundata, element, comp_manns, out=False):
    # create .data files from parameters in setrun.py
    rundata.write()
    # Run the code from the .data files
    nbtools.make_output_and_plots(label="_{}".format(element), verbose=out)
    print("{:.0%} complete".format((comp_manns.index(element) + 1) / (len(comp_manns))))

## Show animation

In [ ]:
def show_anim(anim, form=5):
    html_version = (
        HTML(anim.to_jshtml()) if form == "js" else HTML(anim.to_html5_video())
    )
    return html_version

## Figures to animate

Concatenate frames to be used in animation, then creating the user-facing animation. 

In [ ]:
def stitch_anim(anim_area, form=5):
    if anim_area == "Full map":
        frames, exno, figno, extra = 5, 1, 0, ""

    elif anim_area == "North west coast":
        frames, exno, figno, extra = 19, 2, 1, 0

    else:
        print("Error : Check the animation area was entered correctly")
    # -------------------------------------------------
    if path.isdir("_plots") == False:
        mkdir("_plots")

    old_frames = glob("_plots/*")
    for f in old_frames:
        remove(f)
    # -------------------------------------------------
    for a in range(exno - 1, frames + 1):
        im = Image.open(
            "_plots_{}/frame{:04d}fig{}.png".format(anim_manns[0], a, figno)
        )
        im_alt = Image.open(
            "_plots_{}/frame{:04d}fig{}.png".format(anim_manns[1], a, figno)
        )
        dst = Image.new("RGB", (im.width + im_alt.width, im.height))
        dst.paste(im, (0, 0))
        dst.paste(im_alt, (im.width, 0))
        dst.save("_plots/frame{:04d}fig{}.png".format(a, figno))
    # -------------------------------------------------
    anim = animation_tools.animate_from_plotdir("_plots", figno)
    # -------------------------------------------------
    anim_path = anim_dir + "/{}".format(anim_area.replace(" ", "_"))
    if path.isdir(anim_path) == False:
        mkdir(anim_path)
        copy_tree("_plots", anim_path)
    print("Figures for animation saved to:", anim_path)

    print(
        "\nMannings n = {} on left animation, and n = {} on right "
        "animation".format(choose_manns[anim_manns[0]], choose_manns[anim_manns[1]])
    )

    return show_anim(anim, form)

## Import gauge data

In [ ]:
def gauge_data():
    from setplot import setplot

    plotdata = setplot()
    # reset variables

    t = [None] * amount_manns
    u = [None] * amount_manns
    v = [None] * amount_manns
    eta = [None] * amount_manns
    h = [None] * amount_manns
    u_d = [None] * amount_manns
    v_d = [None] * amount_manns
    hu = [None] * amount_manns
    hv = [None] * amount_manns
    dis_lat = [None] * amount_manns
    dis_lon = [None] * amount_manns
    # -------------------------------------------------
    for k in range(amount_manns):
        plotdata.outdir = "_output_{}".format(k + 1)
        # get gauge data
        g = plotdata.getgauge(gauge_num)
        t[k] = g.t / 3600.0  # convert to hours
        u[k] = g.q[1, :]  # u = east-west wave velocity
        v[k] = g.q[2, :]  # v = north-south wave velocity
        eta[k] = g.q[3, :]  # eta = h + B (depth plus bathymetry)
        h[k] = g.q[0, :]  # water depth
        u_d[k] = g.q[1, :] / h[k]  # u_d = east-west depth-averaged velocity
        v_d[k] = g.q[2, :] / h[k]  # v_d = north-south depth-averaged velocity
        hu[k] = g.q[1, :] * h[k]  # u = east-west wave discharge
        hv[k] = g.q[2, :] * h[k]  # v = north-south wave discharge

        if plot_unit == "km/h" and gauge_type == "stationary":
            u[k], v[k] = (u[k] * 3.6), (v[k] * 3.6)
            hu[k], hv[k] = (hu[k] * 3.6), (hv[k] * 3.6)

    if gauge_type == "lagrangian":

        for j in range(amount_manns):
            dis_lat[j] = empty([len(u[j])])
            dis_lon[j] = empty([len(v[j])])
            for i in range(len(u[j])):
                dis_lat[j][i] = (
                    det[list(dict_gauge).index(gauge_name)][1] - u[j][i]
                ) * 110045  # number is the approximate length in metres of a degree of latitude
            for h in range(len(v[j])):
                dis_lon[j][h] = (
                    det[list(dict_gauge).index(gauge_name)][2] - v[j][h]
                ) * 87870.18  # --""-- longitude, both found on https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps

    return t, u, v, eta, u_d, v_d, hu, hv, dis_lat, dis_lon

## Parameterise graph

In [ ]:
def plotter_details(measure):
    if measure == "height":
        var = eta
        yunit = "m"
        heading = "Sea surface elevation"

    elif measure == "DAvelocity":
        var = [u_d, v_d]
        yunit = "m/s" if plot_unit == "m/s" else "km/h"
        heading = "Depth-averaged velocity"

    elif measure == "Wvelocity":
        var = [u, v]
        yunit = "m\u00B2/s" if plot_unit == "m/s" else "km\u00B2/h"
        heading = "Wave velocity"

    elif measure == "discharge":
        var = [hu, hv]
        yunit = "m\u00B3/s" if plot_unit == "m/s" else "km\u00B3/h"
        heading = "Discharge"

    elif measure == "displacement":
        var = [dis_lat, dis_lon]
        yunit = "m"
        heading = "Displacement"

    direct = ["East-west", "North-south"]

    return var, direct, yunit, heading

## Plot graph

In [ ]:
def plotter(measure):
    if gauge_type == "lagrangian" and measure != "displacement":
        print(
            'skipping plot, as gauge type must be "stationary" to plot {}'.format(
                measure
            )
        )
        skip = True
    elif gauge_type == "stationary" and measure == "displacement":
        print(
            'skipping plot, as gauge type must be "lagrangian" to plot {}'.format(
                measure
            )
        )
        skip = True
    else:
        skip = False
    # -------------------------------------------------
    if skip == False:
        var, direct, yunit, heading = plotter_details(measure)

        if measure == "height":
            grid()
            for k in range(amount_manns):
                plot(
                    t[k],
                    var[k],
                    linestyle=line[k][0],
                    color=line[k][1],
                    label="{}".format(manns[k]),
                )
            if plot_legend == True:
                legend(title="Manning's n values")
            title("{} at buoy {}".format(heading, gauge_num), fontsize="x-large")
            xlabel("Time since earthquake (h)")
            ylabel("{} ({})".format(heading, yunit))

        else:
            if gauge_type == "lagrangian":
                suptitle("{} of buoy {}".format(heading, gauge_num), fontsize="x-large")
            else:
                suptitle("{} at buoy {}".format(heading, gauge_num), fontsize="x-large")
            for l in range(2):
                subplot(1, 2, (l + 1))
                grid()
                for k in range(amount_manns):
                    plot(
                        t[k],
                        var[l][k],
                        linestyle=line[k][0],
                        color=line[k][1],
                        label="{}".format(manns[k]),
                    )
                if plot_legend == True:
                    legend(title="Manning's n values")
                title("{} direction".format(direct[l]))
                xlabel("Time since earthquake (h)")
                ylabel("{} ({})".format(heading, yunit))
                ticklabel_format(
                    style="sci", axis="y", scilimits=(-4, 5), useMathText=True
                )

            if gauge_type == "lagrangian":
                ticklabel_format(useOffset=False, style="plain")

        tight_layout()
        gcf().set_size_inches(14, 6)
        savefig(
            "_saved/_plots/manns_{}/{}-gauge_{}.png".format(
                manns, gauge_name.replace(" ", "_"), measure
            )
        )
        show()
        print("\n")

## Plot comparison graph

Plots the graph which compares the simulated data with the real data of **gauge 32412**

In [ ]:
def plotter_compare():
    if gauge_type == "lagrangian":
        print('Comparison can only be performed on "stationary" guage type')
    else:
        ax = gca()

        xmin, xmax, ymin, ymax = (3, 7, -0.20, 0.30)

        for k in range(amount_manns):
            if plot_legend == True:
                ax.plot(
                    t[k],
                    eta[k],
                    linestyle=line[k][0],
                    color=line[k][1],
                    label="{}".format(manns[k]),
                )
            else:
                ax.plot(t[k], eta[k], linestyle=line[k][0], color=line[k][1])
        ax.plot(0, 0, color="k", label="Real life data")
        ax.legend(title="Manning's n values") if plot_legend == True else ax.legend()
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_xlabel("Time since earthquake (h)")
        ax.set_ylabel("Sea surface elevation (m)")
        ax.set_title("Sea surface elevation at buoy 32412", fontsize="x-large")

        ax.set_zorder(2)
        ax.set_facecolor("none")

        ax_tw_x = ax.twinx()
        ax_tw_x.axis("off")
        ax2 = ax_tw_x.twiny()

        im = imread("gauge32412-actual-detide.png")
        ax2.imshow(im, extent=[xmin, xmax, ymin, ymax], aspect="auto")
        ax2.axis("off")

        gcf().set_size_inches(14, 6)
        savefig(
            "_saved/_plots/manns_{}/{}-compare_gauge_height.png".format(
                manns, gauge_name.replace(" ", "_")
            )
        )
        show()

# Custom parameters

## Manning's values

If `True`, Manning values will be a family, if `False`, they will be custom

In [ ]:
# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
family_mann = False
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

### Family of Manning's values

Assign Manning's values as a family, spaced out and centered around a middle value. 

Please let the amount of Manning's values be no less than `2`.

Note : Each value may take 1-2 minutes to plot dependent on your machine.

In [ ]:
# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
amount_manns = 5  # Amount of Manning's values to create
mid_mann = 0.8  # Middle Manning's value
space_manns = 0.2  # Space between each Manning's value. I.e, to plot 3 Manning's values, with middle value 0.05, and space value 0.02/
# The total created Manning's values would be [0.03, 0.05, 0.07]
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

### Custom Manning's values

Dictionary of custom mannings values.

To keep a value in the dictionary, but exclude it from simulation: place a `#` in front of the line it is on (This transforms the line to a comment, so the program won't 'read' it).

Format : 'Description/name' : Manning's n value

In [ ]:
# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
dict_mann = {
    "test": 5,
    "geoclaw": 0.025,
    #     "kotani_coast": 0.025,
    #     "kotani_farmland": 0.02,
    #     "kotani_forest": 0.03,
    #     "kotani_low_urban": 0.04,
    #     "kotani_med_urban": 0.06,
    #     "kotani_high_urban": 0.08,
    #     "yanigasawa_mangrove": 0.04,
    #     "yanigasawa_vegitation": 0.05,
    #     'yanigasawa_buildings':0.06,
    #     "koshimura_low_urban": 0.053,
    #     "koshimura_med_urban": 0.094,
    #     "koshimura_high_urban": 0.172,
}
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

## Gauges

Dictionary of custom gauges.

To keep a value in the dictionary, but exclude it from simulation: place a `#` in front of the line it is on (This transforms the line to a comment, so the program won't 'read' it).

Entries are of the following form (try to keep location description under 35 characters, and keep *latitude* and *longitude* from 1-3 decimal places to avoid float miscalculations when plotting):

'location description':[number(0-999) , latitide, longitude, time start(default to 0.), time steps(default to 1.e10)]

hint: area wave resides in first hour : [-85,-72,-38,-25]

The land border is not the same as that on google maps, so it may be a case of trial and error to show your desired gauge if you are trying to plot very near the coast. For example: The Constitución coast gauge is 6km west of the coast coordinates given on google maps. I have found that plotting 5km or closer often ends in failure to plot velocity, as the height at this gauge is given as 0 (i.e the area being plotted on is land).

In [ ]:
# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
dict_gauge = {
    "buoy 123": [123, -77.3, -12.3, 3.5 * 3600, 1.0e10],
    "buoy 32412": [32412, -86.392, -17.975, 3 * 3600, 1.0e10],
    #     "a far point": [2, -119.999, -0.001, 0.0, 1.0e10],
    #     "Alejandro Selkirk Island west coast": [3, -80.757, -33.768, 0.0, 1.0e10],
    #     "Robinson Crusoe Island west coast": [4, -78.771, -33.669, 0.0, 1.0e10],
    #     "the epicentre": [1, -72.733, -34.909, 0.0, 1.0e10],
    #     "Constitución coast (6km west)": [6, -72.500, -35.331, 0.0, 1.0e10],
}
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

gauge_types is a list of gauge types, which can either be `["stationary"]`, or `["lagrangian"]`, or both `["stationary", "lagrangian"]`. Note: stationary gauges only plot velocity, height and discharge; lagrangian gauges only plot displacement.

Plotting all gauge types takes twice as long as plotting either `"lagrangian"` or `"stationary"` alone. For 10 manning's coefficients, expect to wait around 20-25 minutes to complete.

In [ ]:
# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
gauge_types = ["stationary", "lagrangian"]
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

## Plot details

Display legend of curves on your plot. Either write a conditional function, or replace with a simple `plot_legend = True`, or `plot_legend = False`

In [ ]:
# vvvvvvvvvvvvvvvv
plot_legend = True
# ^^^^^^^^^^^^^^^^

Choose what units the `"Wvelocity"` and `"discharge"` plots will be displayed in. Chooese from either `"m/s"` or `"km/h"`

In [ ]:
# vvvvvvvvvvvvvvvvvvvvv
plot_unit = "km/h"
# ^^^^^^^^^^^^^^^^^^^^^

Choose line colours (define in hex form)

In [ ]:
standard_colour = [
    "#8c510a",
    "#d8b365",
    "#5ab4ac",
    "#01665e",
]  # created at https://colorbrewer2.org/#type=diverging&scheme=BrBG&n=6

# Optionally, see what this plot would look as a person with a color vision deficiency:
Protanopia = ["#72711b", "#c7c777", "#8081ad", "#2c2d5f"]
Deuteranopia = ["#757a1f", "#cacc7c", "#7b75ae", "#261f60"]
Tritanopia = ["#89282b", "#d6868a", "#5eafaf", "#066161"]
Achromatopsia = ["#5a5a5a", "#b5b5b5", "#989898", "#464646"]
Protanomaly = ["#816412", "#d1bf6e", "#6a96ad", "#13445f"]
Deuteranomaly = ["#806014", "#d0bc70", "#6c9cad", "#154b5f"]
Tritanomaly = ["#8a3e16", "#d69e73", "#5cb1ad", "#04635f"]
Achromatomaly = ["#715635", "#c5b490", "#7ba4a1", "#275551"]
# created at https://coolors.co/8c510a-d8b365-5ab4ac-01665e

# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
colour_option = standard_colour
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# Initialise parameter choice

Note : All manning values are shown in their absolute value (the program does not display negative values any differently from positive ones)

In [ ]:
if family_mann == False:
    leave = 0
    for key in dict_mann:
        for keys in dict_mann:
            if (dict_mann[key] == dict_mann[keys]) and (key != keys):
                input(
                    "'{}' and '{}' have the same Manning's n value: {}.\n"
                    "Either:\n"
                    "1. Comment out the duplicate; interrupt, and restart the kernel."
                    " Then rerun the notebook.\n"
                    "2. Press the 'Enter' key to ignore all duplicates"
                    " (Warning: the run-time of the program will be longer than neccessary).\n".format(
                        key, keys, dict_mann[key]
                    )
                )
                leave = 1
            if leave == 1:
                break
        if leave == 1:
            break

In [ ]:
choose_manns = {}

if family_mann == True:
    manns = [None] * amount_manns

elif family_mann == False:
    manns = [None] * len(dict_mann)
    for key in dict_mann:
        k = list(dict_mann).index(key)
        manns[k] = dict_mann[key]
    manns.sort()

amount_manns = len(manns)

for k in range(amount_manns):
    if family_mann == True:

        deci_point = [decimal.Decimal(str(mid_mann)), decimal.Decimal(str(space_manns))]
        roundto = -max(
            [deci_point[0].as_tuple().exponent, deci_point[1].as_tuple().exponent]
        )
        if amount_manns % 2 == 0:
            manns[k] = abs(mid_mann + (k + 1 - (amount_manns) / 2) * space_manns)
        else:
            manns[k] = abs(mid_mann + (k + 1 - (amount_manns + 1) / 2) * space_manns)
        manns[k] = round(manns[k], roundto)
    choose_manns[k + 1] = manns[k]

In [ ]:
det = [None] * len(dict_gauge)

for key in dict_gauge:
    k = list(dict_gauge).index(key)
    det[k] = dict_gauge[key]

# Show additional parameters

Some additional notebook parameters (which are able to be changed in the setrun.py file).

In [ ]:
from setrun import setrun

showdata = setrun()

# -------------------------------------------------
show_cfl_desired = showdata.clawdata.cfl_desired
show_cfl_max = showdata.clawdata.cfl_max
if showdata.clawdata.order == 1:
    show_order = "Godunov"
else:
    show_order = "Lax-Wendroff plus limiters"

if showdata.clawdata.transverse_waves == 0:
    show_transverse_waves = "Donor cell"
elif showdata.clawdata.transverse_waves == 1:
    show_transverse_waves = "Corner Transport Upwind"
else:
    show_transverse_waves = "Corner Transport Upwind + corrections"

show_num_waves = showdata.clawdata.num_waves

show_limiter = (
    str(showdata.clawdata.limiter).replace("'", "").replace("[", "").replace("]", "")
)


show_waves = "F-waves" if showdata.clawdata.use_fwaves == True else "Classic waves"

show_frac_step = showdata.clawdata.source_split
show_coriolis = str(showdata.geo_data.coriolis_forcing)
show_fric_depth = "{:.2e}".format(showdata.geo_data.friction_depth)

# -------------------------------------------------
table_data = [
    ["Desired Courant number", show_cfl_desired],
    ["Maximum Courant number", show_cfl_max],
    ["Riemann solver", show_order],
    ["Wave propogation method", show_transverse_waves],
    ["Number of waves in Riemann Solution", show_num_waves],
    ["Limiters", show_limiter],
    ["Wave type", show_waves],
    ["Fractional step algorithm", show_frac_step],
    ["Coriolis force", show_coriolis],
    ["Friction depth", show_fric_depth],
]
layout = "{: <40}" * 2
for row in table_data:
    print(layout.format(*row))
    print("-" * 80)

# Animations

If you only wish to run the **Plots** section of the notebook, you can comment out everything in the animation section without affecting the rest of the notebook.

## Animation set up

In [ ]:
anim_manns = [None] * 2
if len(choose_manns) > 2:
    anim_manns[0] = int(
        input(
            "Choose (1-{}) the first mannings values to"
            " compare via animation: {}\n".format(amount_manns, choose_manns)
        )
    )
    anim_manns[1] = int(
        input(
            "Choose (1-{}) the second mannings values to"
            " compare via animation: {}\n".format(amount_manns, choose_manns)
        )
    )
else:
    print("Mannings values to be compiled : {}\n".format(manns))
    anim_manns[0] = 1
    anim_manns[1] = 2

anim_dir = "_saved/_anim/manns_[{},{}]".format(
    choose_manns[anim_manns[0]], choose_manns[anim_manns[1]]
)
if path.isdir(anim_dir) == False:
    mkdir(anim_dir)

## North west coast

If the animation window is largely taken up by yellow space: check `"buoy 123"` and `"buoy 32412"` are the first and second entry in `dict_gauge`. If they are not, change the order of the gauges in `dict_gauge` to reflect this and rerun the notebook.

In [ ]:
init_comp(2, anim123 = True)
stitch_anim('North west coast', 'js')

## Full map

In [ ]:
init_comp(1)
stitch_anim('Full map', 5)

# Plots

## Plot set-up

In [ ]:
line_option = ["-", ":", "--", "-."]

line = [None] * amount_manns

for k in range(amount_manns):
    line[k] = [None] * 2
    j = k % len(colour_option)
    l = int(str(k / len(colour_option))[:1])
    line[k][0] = line_option[l]
    line[k][1] = colour_option[j]

In [ ]:
plots_dir = "_saved/_plots/manns_{}".format(manns)
if path.isdir(plots_dir) == True:
    input(
        "Warning: A directory for these manning's values already exists at {}\n"
        "Either :\n"
        "1. Press the 'Enter' key to continue creating the plots (and overwrite the plots in the existing directory).\n"
        "2. Check the directory does not already contain the plots you wanted to create."
        " If so, there is no need to run the notebook as it is presently configured.\n".format(
            plots_dir
        )
    )
else:
    mkdir(plots_dir)

Display an image of the map with custom gauges to check positioning. Process is faster than running one of the animations above, as it runs for a single Manning's coefficient.

In [ ]:
init_comp(1, plots=True, gauge_map=True)
ipyImage(filename="_plots_1/frame0000fig0.png")

## Plot gauges

In [ ]:
print("\nAll plots will be saved to {}\n".format(plots_dir))
for gauge_type in gauge_types:
    init_comp(2, plots=True, gauge_type=gauge_type)
    for key in dict_gauge:
        gauge_name = key
        gauge_num = dict_gauge[gauge_name][0]
        print("-" * 100)
        t, u, v, eta, u_d, v_d, hu, hv, dis_lat, dis_lon = gauge_data()
        print("-" * 100)
        gauge_label = "Gauge: {}".format(gauge_name)
        label_spacer = " " * (60 - len(gauge_label))
        print(
            "\n{}{}\n{}{}\n".format(
                label_spacer, gauge_label, label_spacer, "=" * len(gauge_label)
            )
        )
        if gauge_type == "stationary":
            plotter("DAvelocity")
            plotter("height")
            if gauge_num == 32412:
                plotter_compare()
            plotter("Wvelocity")
            plotter("discharge")
        if gauge_type == "lagrangian":
            plotter("displacement")